# To clean the data a bit

First, remove the columns we don't need, and also the data points where 0 is traded

In [1]:
import pandas as pd

In [6]:
data = pd.read_csv("EU_trade_electricity.csv")

data.head()

,DATAFLOW,LAST UPDATE,freq,siec,partner,unit,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG,CONF_STATUS
0,ESTAT:NRG_TI_EH(1.0),24/04/25 23:00:00,Annual,Electricity,Austria,Gigawatt-hour,Albania,2018,0.0,NaN,NaN
1,ESTAT:NRG_TI_EH(1.0),24/04/25 23:00:00,Annual,Electricity,Austria,Gigawatt-hour,Albania,2019,0.0,NaN,NaN
2,ESTAT:NRG_TI_EH(1.0),24/04/25 23:00:00,Annual,Electricity,Austria,Gigawatt-hour,Albania,2020,0.0,NaN,NaN
3,ESTAT:NRG_TI_EH(1.0),24/04/25 23:00:00,Annual,Electricity,Austria,Gigawatt-hour,Albania,2021,0.0,NaN,NaN
4,ESTAT:NRG_TI_EH(1.0),24/04/25 23:00:00,Annual,Electricity,Austria,Gigawatt-hour,Albania,2022,0.0,NaN,NaN


In [ ]:

data = data.filter(items=["partner", "geo", "TIME_PERIOD", "OBS_VALUE"])
data = data[data["OBS_VALUE"] > 0]
data.head()

,partner,geo,TIME_PERIOD,OBS_VALUE
36,Austria,Czechia,2018,112.561
37,Austria,Czechia,2019,144.762
38,Austria,Czechia,2020,378.420
39,Austria,Czechia,2021,197.000
40,Austria,Czechia,2022,37.188


In [10]:
data.to_csv("cleaned_EU_trade.csv", index=False)